In [ ]:
import numpy as np
import pandas as pd

from database import Movie
from database import Rating
from recommender import NearestNeighbor

np.random.seed(42)
pd.set_option('precision', 1)

In [ ]:
movie_database = Movie.load()
rating_database = Rating.load(nrows=10**6 // 2)
train_database, _ = rating_database.split()

users = rating_database.data.groupby('userId')['movieId'].agg('count')

user = np.random.choice(users[users > 10].index)
user_ratings = rating_database.find_by_user(user)
user_movies = movie_database.find(user_ratings.index).join(user_ratings)

# User-based Filtering

In [ ]:
model = NearestNeighbor()
model.fit((train_database.data['userId'],
           train_database.data['movieId'],
           train_database.data['rating']))

estimates = [model.predict(user, movie) for movie in user_movies.index]
user_movies.join(pd.DataFrame({'estimate': estimates}, index=user_movies.index))

# Item-based Filtering

In [ ]:
model = NearestNeighbor()
model.fit((train_database.data['movieId'],
           train_database.data['userId'],
           train_database.data['rating']))

estimates = [model.predict(movie, user) for movie in user_movies.index]
user_movies.join(pd.DataFrame({'estimate': estimates}, index=user_movies.index))